In [10]:
pip install chromadb sentence-transformers google-generativeai python-dotenv

In [11]:
from sentence_transformers import SentenceTransformer
import chromadb
import json
import os

In [2]:
data = json.load(open('/content/data.json'))

In [3]:
data

{'users': [{'id': '1',
   'name': 'amr',
   'email': 'a@gmail.com',
   'password': '123456'},
  {'id': 2,
   'name': 'amrt belal',
   'email': 'amrbelal852@gmail.com',
   'password': '222222222',
   'username': 'd'},
  {'id': 3,
   'name': 'ali',
   'email': 'amrbelal885522@gmail.com',
   'password': '11111'},
  {'id': 4,
   'name': 'Amr Belal',
   'email': 'amrbelal852@gmail.com',
   'password': '3333333333333'},
  {'id': 'ef08',
   'name': 'alii',
   'email': 'amrbelal885522@gmail.com',
   'password': '222222222222'}],
 'products': [{'id': 1,
   'name': 'Coco Mademoiselle',
   'description': 'A blend of citrus, jasmine, rose, and patchouli with a hint of vanilla.',
   'perfume_type': 'Eau de Parfum',
   'price': 120,
   'perfume_category': 'Floral',
   'image_link': 'https://images.unsplash.com/photo-1708733145706-82da0d0596e9?q=80&w=736&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D',
   'rating': 4.8,
   'count': 150,
   'category': 'w

In [4]:
data["products"]

[{'id': 1,
  'name': 'Coco Mademoiselle',
  'description': 'A blend of citrus, jasmine, rose, and patchouli with a hint of vanilla.',
  'perfume_type': 'Eau de Parfum',
  'price': 120,
  'perfume_category': 'Floral',
  'image_link': 'https://images.unsplash.com/photo-1708733145706-82da0d0596e9?q=80&w=736&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D',
  'rating': 4.8,
  'count': 150,
  'category': 'women'},
 {'id': 2,
  'name': 'Creed Aventus',
  'description': 'A mix of pineapple, birch, musk, and oakmoss for a bold and fresh scent.',
  'perfume_type': 'Eau de Parfum',
  'price': 280,
  'perfume_category': 'Woody',
  'image_link': 'https://images.unsplash.com/photo-1705338670422-01133208eab9?q=80&w=764&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D',
  'rating': 4.7,
  'count': 120,
  'category': 'men'},
 {'id': 3,
  'name': 'Chanel No. 5',
  'description': 'Features aldehydes, ylang-ylang, 

In [5]:
products = data["products"]

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
from typing import Collection
db_path = "/content/drive/MyDrive/perfume_vector_db"

client = chromadb.PersistentClient(path=db_path)


collection = client.get_or_create_collection(
    name="perfume_products",
    metadata={"hnsw:space": "cosine"}
)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
texts = []
metadatas = []
ids = []

for i,p in enumerate(products):
  text = f"{p['name']} {p['perfume_type']} {p['description']} {p['perfume_category']} {p['price']} "
  texts.append(text)
  metadatas.append(p)
  ids.append(f"product_{i}_{p['name'].replace(' ', '_')}")

collection.add(
  documents=texts,
  metadatas=metadatas,
  ids=ids
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 32.2MiB/s]


In [17]:
query = "perfume with jasmine and vanilla"
results = collection.query(
    query_texts=[query],
    n_results=2,
    include=["documents", "metadatas", "distances"]
)

for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
    print(f"🔹 {meta['name']} by {meta['perfume_type']} - Score: {1 - dist:.2f}")


🔹 Coco Mademoiselle by Eau de Parfum - Score: 0.60
🔹 Yves Saint Laurent Libre by Eau de Parfum - Score: 0.59
